In [1]:
!pip install -q pymongo rich pandas

In [2]:
import pymongo
import urllib.parse
from pymongo import MongoClient
from bson.objectid import ObjectId
import datetime
from rich import print
import pandas as pd
import os

In [3]:
username = urllib.parse.quote_plus(os.environ["MONGO_INITDB_ROOT_USERNAME"])
password = urllib.parse.quote_plus(os.environ["MONGO_INITDB_ROOT_PASSWORD"])
mongo_host = "mongodb"
# client = MongoClient('mongodb://%s:%s@mongodb' % (username, password))
client = MongoClient(f"mongodb://{username}:{password}@{mongo_host}")

In [4]:
db = client['tarp-data']
collection = db['tarp-params']

In [5]:
param = { "parameter": 1000000,
         "source":
            {
                "model": "==GLWM",
                "run": 1639742400,
                "forecast": 3600
            },
         "name": "Temperature Air",
         "unit": "kelvin",
         "levels": 
            {
                "2m": [270, 275, 280, 285],
                "900mb": [250, 240, 230, 220]
            },
         "time_entered": datetime.datetime.utcnow(),
         "metadata":
            {
                "edited_by": "Jason",
                "tags": ["some", "tags", "here"]
            }
       }

param2 = { "parameter": 500851,
         "source":
            {
                "model": "==GLWM",
                "run": 1639742400,
                "forecast": 3600
            },
         "name": "Relative Humidity",
         "unit": "percent",
         "levels": 
            {
                "2m": [60, 65, 70, 75],
                "900mb": [10, 15, 16, 17]
            },
         "time_entered": datetime.datetime.utcnow(),
         "metadata":
            {
                "edited_by": "Jason",
                "tags": ["different", "tags", "here", "too"]
            }
       }

In [6]:
object_id = collection.insert_one(param).inserted_id
object_id2 = collection.insert_one(param2).inserted_id

In [7]:
print(object_id)
print(object_id2)

61bcf1e60d5e6b0870dc0728

61bcf1e60d5e6b0870dc0729

In [8]:
print(collection.find_one(object_id))
print(collection.find_one(object_id2))

{
    '_id': ObjectId('61bcf1e60d5e6b0870dc0728'),
    'parameter': 1000000,
    'source': {'model': '==GLWM', 'run': 1639742400, 'forecast': 3600},
    'name': 'Temperature Air',
    'unit': 'kelvin',
    'levels': {'2m': [270, 275, 280, 285], '900mb': [250, 240, 230, 220]},
    'time_entered': datetime.datetime(2021, 12, 17, 20, 24, 6, 431000),
    'metadata': {'edited_by': 'Jason', 'tags': ['some', 'tags', 'here']}
}

{
    '_id': ObjectId('61bcf1e60d5e6b0870dc0729'),
    'parameter': 500851,
    'source': {'model': '==GLWM', 'run': 1639742400, 'forecast': 3600},
    'name': 'Relative Humidity',
    'unit': 'percent',
    'levels': {'2m': [60, 65, 70, 75], '900mb': [10, 15, 16, 17]},
    'time_entered': datetime.datetime(2021, 12, 17, 20, 24, 6, 431000),
    'metadata': {'edited_by': 'Jason', 'tags': ['different', 'tags', 'here', 'too']}
}

In [9]:
type(collection.find_one(object_id))

dict

In [10]:
print(collection.find_one(object_id).get('time_entered'))
print(type(collection.find_one(object_id).get('time_entered')))

2021-12-17 20:24:06.431000

<class 'datetime.datetime'>

In [13]:
cursor = collection.find()
data = list(cursor)
df = pd.DataFrame(data)

In [14]:
df

,_id,parameter,source,name,unit,levels,time_entered,metadata
0,61bce001d1524bb34deca366,1000000,"{'model': '==GLWM', 'run': 1639742400, 'foreca...",Temperature Air,kelvin,"{'2m': [270, 275, 280, 285], '900mb': [250, 24...",2021-12-17 19:07:45.582,"{'edited_by': 'Jason', 'tags': ['some', 'tags'..."
1,61bce001d1524bb34deca367,500851,"{'model': '==GLWM', 'run': 1639742400, 'foreca...",Relative Humidity,percent,"{'2m': [60, 65, 70, 75], '900mb': [10, 15, 16,...",2021-12-17 19:07:45.582,"{'edited_by': 'Jason', 'tags': ['different', '..."
2,61bcf1d628377163c89cfe40,1000000,"{'model': '==GLWM', 'run': 1639742400, 'foreca...",Temperature Air,kelvin,"{'2m': [270, 275, 280, 285], '900mb': [250, 24...",2021-12-17 20:23:50.930,"{'edited_by': 'Jason', 'tags': ['some', 'tags'..."
3,61bcf1d628377163c89cfe41,500851,"{'model': '==GLWM', 'run': 1639742400, 'foreca...",Relative Humidity,percent,"{'2m': [60, 65, 70, 75], '900mb': [10, 15, 16,...",2021-12-17 20:23:50.930,"{'edited_by': 'Jason', 'tags': ['different', '..."
4,61bcf1e60d5e6b0870dc0728,1000000,"{'model': '==GLWM', 'run': 1639742400, 'foreca...",Temperature Air,kelvin,"{'2m': [270, 275, 280, 285], '900mb': [250, 24...",2021-12-17 20:24:06.431,"{'edited_by': 'Jason', 'tags': ['some', 'tags'..."
5,61bcf1e60d5e6b0870dc0729,500851,"{'model': '==GLWM', 'run': 1639742400, 'foreca...",Relative Humidity,percent,"{'2m': [60, 65, 70, 75], '900mb': [10, 15, 16,...",2021-12-17 20:24:06.431,"{'edited_by': 'Jason', 'tags': ['different', '..."


In [15]:
pd.json_normalize(data)

,_id,parameter,name,unit,time_entered,source.model,source.run,source.forecast,levels.2m,levels.900mb,metadata.edited_by,metadata.tags
0,61bce001d1524bb34deca366,1000000,Temperature Air,kelvin,2021-12-17 19:07:45.582,==GLWM,1639742400,3600,"[270, 275, 280, 285]","[250, 240, 230, 220]",Jason,"[some, tags, here]"
1,61bce001d1524bb34deca367,500851,Relative Humidity,percent,2021-12-17 19:07:45.582,==GLWM,1639742400,3600,"[60, 65, 70, 75]","[10, 15, 16, 17]",Jason,"[different, tags, here, too]"
2,61bcf1d628377163c89cfe40,1000000,Temperature Air,kelvin,2021-12-17 20:23:50.930,==GLWM,1639742400,3600,"[270, 275, 280, 285]","[250, 240, 230, 220]",Jason,"[some, tags, here]"
3,61bcf1d628377163c89cfe41,500851,Relative Humidity,percent,2021-12-17 20:23:50.930,==GLWM,1639742400,3600,"[60, 65, 70, 75]","[10, 15, 16, 17]",Jason,"[different, tags, here, too]"
4,61bcf1e60d5e6b0870dc0728,1000000,Temperature Air,kelvin,2021-12-17 20:24:06.431,==GLWM,1639742400,3600,"[270, 275, 280, 285]","[250, 240, 230, 220]",Jason,"[some, tags, here]"
5,61bcf1e60d5e6b0870dc0729,500851,Relative Humidity,percent,2021-12-17 20:24:06.431,==GLWM,1639742400,3600,"[60, 65, 70, 75]","[10, 15, 16, 17]",Jason,"[different, tags, here, too]"
